In [ ]:
import hydra
import pytorch_lightning as pl
import torch
from pathlib import Path
import albumentations as A
from src.transforms.albumentations_adapter import Adapter
import src.utils.bbox_inverse_transform
import json

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
prediction_dir_path = Path('/datagrid/personal/kuntluka/pretrained/dataset_4000')

In [ ]:
for file in list(prediction_dir_path.iterdir()):
    print(file)
    predict_from_ckpt(file)

In [ ]:
def create_predictions_name(ckpt_name):
    base_name = ckpt_name.split('_0.')[0]
    name = 'predictions_' + base_name + '.json'
    return name

def create_overrides(ckpt_name):
    overrides = []
    if 'retinanet' in ckpt_name:
        overrides.append('module=retinanet')

    elif 'faster_rcnn' in ckpt_name:
        overrides.append('module=faster_rcnn')

    elif 'yolov5' in ckpt_name:
        overrides.append('module=yolov5')
    
    elif 'efficientdet' in ckpt_name:
        overrides.append('module=efficientdet')
    
    if 'medium' in ckpt_name:
        overrides.append('module.backbone=medium_p6')

    elif 'large' in ckpt_name:
        overrides.append('module.backbone=large_p6')

    elif 'resnet101' in ckpt_name:
        overrides.append('module.backbone=resnet101_fpn_1x')

    elif 'swint' in ckpt_name:
        overrides.append('module.backbone=swin_t_p4_w7_fpn_1x_coco')

    return overrides

In [ ]:
def predict_from_ckpt(ckpt_path : Path):
    ckpt_name = ckpt_path.name
    overrides = create_overrides(ckpt_name)
    pred_name = create_predictions_name(ckpt_name)
    overrides.append('logger.wandb.project=inference')

    with hydra.initialize(config_path="configs"):
        cfg = hydra.compose(
            config_name="train",
            overrides=overrides
        )
    model = hydra.utils.instantiate(cfg.module.model)

    model.load_state_dict(torch.load(ckpt_path, map_location='cpu')['state_dict'])
    logger = []
    if "logger" in cfg:
        for _, lg_conf in cfg.logger.items():
            if "_target_" in lg_conf:
                logger.append(hydra.utils.instantiate(lg_conf))

    statistics = dict(mean=0.3669, std=0.2768)
    trainer = hydra.utils.instantiate(cfg.trainer, logger=logger)
    v = Adapter([A.Resize(896,1024), A.Normalize(mean=statistics["mean"], std=statistics["std"])])
    dm = hydra.utils.instantiate(cfg.datamodule, train_transforms=v, val_transforms=v)
    dm.setup()

    test_prediction = trainer.predict(model, dm.predict_dataloader('test'))
    val_prediction = trainer.predict(model, dm.predict_dataloader('val'))
    train_prediction = trainer.predict(model, dm.predict_dataloader('train'))

    test_data = src.utils.bbox_inverse_transform.predictions_to_fiftyone(test_prediction, stage='test')
    val_data = src.utils.bbox_inverse_transform.predictions_to_fiftyone(val_prediction, stage='val')
    train_data = src.utils.bbox_inverse_transform.predictions_to_fiftyone(train_prediction, stage='train')
    data = {**test_data, **val_data, **train_data}

    with open(pred_name, 'w') as f:
        json.dump(data, f)


In [ ]:

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="3"

with hydra.initialize(config_path="configs"):
    cfg = hydra.compose(
        config_name="train",
        overrides=['module=yolov5', 'module.backbone=small_p6', 'datamodule.batch_size=12']
    )
model = hydra.utils.instantiate(cfg.module.model)

statistics = dict(mean=0.3669, std=0.2768)
trainer = hydra.utils.instantiate(cfg.trainer)
v = Adapter([A.Resize(896,1024), A.Normalize(mean=statistics["mean"], std=statistics["std"])])
dm = hydra.utils.instantiate(cfg.datamodule, train_transforms=v, val_transforms=v)

In [ ]:
lr_finder = trainer.tuner.lr_find(model, datamodule=dm)

In [ ]:
lr_finder.plot()

In [ ]:
plt.show()

In [ ]:

trainer = hydra.utils.instantiate(cfg.trainer, logger=logger)